In [1]:
import os
print("SPARK_CONF_DIR =", os.environ.get("SPARK_CONF_DIR"))
print("SPARK_HOME =", os.environ.get("SPARK_HOME"))

SPARK_CONF_DIR = /opt/spark/conf
SPARK_HOME = /opt/spark


In [2]:
conf_path = "/opt/spark/conf/spark-defaults.conf"
with open(conf_path, 'r') as f:
    lines = f.readlines()
print("First 10 lines of spark-defaults.conf:", lines[:10])

First 10 lines of spark-defaults.conf: ['# ==============================================================================\n', '# SPARK CONFIGURATION FOR POLARIS + ICEBERG + MINIO\n', '# ==============================================================================\n', '\n', '# Basic Extensions\n', '# https://iceberg.apache.org/docs/latest/spark-configuration\n', 'spark.sql.extensions                   org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions\n', '\n', '# Catalog Configuration (The Polaris Connection)\n', '# https://iceberg.apache.org/docs/latest/spark-configuration\n']


In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Polaris Debug")
    .getOrCreate()
)

[07:49:42] [Thread-3] WARN  org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Test
print(spark.conf.get("spark.sql.catalog.polaris.uri"))

http://polaris:8181/api/catalog


In [5]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS polaris.olist_raw")

[07:49:51] [Thread-3] INFO  org.apache.iceberg.rest.auth.AuthManagers - Loading AuthManager implementation: org.apache.iceberg.rest.auth.OAuth2Manager
[07:49:53] [Thread-3] INFO  org.apache.iceberg.CatalogUtil - Loading custom FileIO implementation: org.apache.iceberg.aws.s3.S3FileIO


DataFrame[]

In [6]:
spark.sql("SHOW NAMESPACES IN polaris").show()

+---------+
|namespace|
+---------+
|olist_raw|
+---------+



In [7]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS polaris.olist_raw.test_connection (
        id INT,
        status STRING
    ) USING iceberg
""")

[07:49:59] [Thread-3] INFO  org.apache.iceberg.CatalogUtil - Loading custom FileIO implementation: org.apache.iceberg.aws.s3.S3FileIO


DataFrame[]

In [8]:
import socket
socket.gethostbyname("minio")  # should return the internal IP

'172.18.0.2'

In [9]:
# Insert data
spark.sql("""
INSERT INTO polaris.olist_raw.test_connection VALUES
(1, 'pending'),
(2, 'complete')
""")

# Read back
spark.sql("SELECT * FROM polaris.olist_raw.test_connection").show()

[07:50:05] [Thread-3] INFO  org.apache.iceberg.spark.source.SparkWrite - Committing append with 2 new data files to table polaris.olist_raw.test_connection
[07:50:06] [Thread-3] INFO  org.apache.iceberg.SnapshotProducer - Committed snapshot 4322786829089748504 (MergeAppend)
[07:50:06] [Thread-3] INFO  org.apache.iceberg.metrics.LoggingMetricsReporter - Received metrics report: CommitReport{tableName=polaris.olist_raw.test_connection, snapshotId=4322786829089748504, sequenceNumber=2, operation=append, commitMetrics=CommitMetricsResult{totalDuration=TimerResult{timeUnit=NANOSECONDS, totalDuration=PT0.705782153S, count=1}, attempts=CounterResult{unit=COUNT, value=1}, addedDataFiles=CounterResult{unit=COUNT, value=2}, removedDataFiles=null, totalDataFiles=CounterResult{unit=COUNT, value=4}, addedDeleteFiles=null, addedEqualityDeleteFiles=null, addedPositionalDeleteFiles=null, addedDVs=null, removedDeleteFiles=null, removedEqualityDeleteFiles=null, removedPositionalDeleteFiles=null, removed

In [10]:
# Check what Spark actually loaded
conf = spark.sparkContext.getConf().getAll()
for item in conf:
    if "s3.endpoint" in item[0]:
        print(f"✅ FOUND: {item}")

✅ FOUND: ('spark.sql.catalog.polaris.s3.endpoint', 'http://minio:9000/')


In [11]:
import boto3

s3 = boto3.client(
    's3',
    endpoint_url='http://minio:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    region_name='us-east-1'
)

print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': '18872923B65F34B5', 'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'accept-ranges': 'bytes', 'content-length': '543', 'content-type': 'application/xml', 'server': 'MinIO', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'x-amz-request-id': '18872923B65F34B5', 'x-content-type-options': 'nosniff', 'x-ratelimit-limit': '2051', 'x-ratelimit-remaining': '2051', 'x-xss-protection': '1; mode=block', 'date': 'Sat, 03 Jan 2026 07:50:12 GMT'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'raw', 'CreationDate': datetime.datetime(2026, 1, 3, 7, 43, 40, 415000, tzinfo=tzlocal())}, {'Name': 'staging', 'CreationDate': datetime.datetime(2026, 1, 3, 7, 43, 40, 447000, tzinfo=tzlocal())}, {'Name': 'warehouse', 'CreationDate': datetime.datetime(2026, 1, 3, 7, 43, 40, 37